## Once we have a classifier, we can run it on MS data to classify MS brain tissue.

In [2]:
import numpy as np 
from sklearn.preprocessing import StandardScaler
from skops.io import dump, load
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
import pandas as pd
import nibabel as nib
import sklearn.metrics as metrics
import skfuzzy as fuzz
import time

## Define folder where data lives right off the bat
base_folder = "/Users/sharada/Documents/Projects/MDDE/V3/SAMPLE/"

### Load in microstructure maps from test datasets (MS subjects) one at a time, classify each dataset based on the saved classifier. 

### Generate clustered maps for each subject. 

### Assumes that all maps are registered to the same space for each subject (e.g. MWF, uFA and CMD maps are registered), and a whole brain mask excluding CSF is present. To do this, see the processing/ folder.

### Also repeat this for the atlases to generate the atlas of tissue classification.

In [3]:
##Do each MS subject one by one. Also repeat for the atlas (file paths may need to change!).
subject_list = ["LOBSTR_P026"]

##The trained model we have saved. Can iterate through a few different models if needed.
trained_models = [base_folder + '/model_6clusters.skops']

##The number of clusters we are going with.
cluster_numbers = [6]

for subject in subject_list:
    ##Set up folder names
    folder_mask = base_folder + subject + "/ants/CALIPR/" ##Location of whole-brain mask excluding CSF
    folder_metrics_mwf = base_folder + subject + "/ants/CALIPR/" ##Location of MWF map
    folder_metrics_tvde = base_folder + subject + "/ants/CALIPR_MDD/" ##Location of uFA, CMD maps (registered to MWF space)
    folder_clustering = base_folder + subject + "/clustering_outputs/" ##Output location
    !mkdir {folder_clustering} ##Make the output folder

    mask = nib.load(folder_mask+"mask_no_csf.nii.gz").get_fdata()  ##Whole brain mask excluding CSF
    img_data_1 = nib.load(folder_metrics_tvde+"ufa.nii.gz").get_fdata()  ##uFA map
    img_data_2 = nib.load(folder_metrics_tvde+"cmd.nii.gz").get_fdata()  ##C_MD map
    img_data_3 = nib.load(folder_metrics_mwf+"MWF_brain_ero.nii.gz").get_fdata()  ##MWF map

    ##Separately, hold on to the mask data in this format as we will need its header later on when we generate an image.
    img_mask = nib.load(folder_mask+"mask_no_csf.nii.gz") 

    ##Wherever the mask is 1, take only that metric data into account.
    img_1_nonzero = img_data_1[mask == 1]
    img_2_nonzero = img_data_2[mask == 1]
    img_3_nonzero = img_data_3[mask == 1]

    print("Classifying: " + subject)

    ##Put all metric data into one cohesive dataframe.
    df_test = pd.DataFrame() 
    df_test['img_1'] = img_1_nonzero
    df_test['img_2'] = img_2_nonzero
    df_test['img_3'] = img_3_nonzero

    ##Scale the test dataset (normalize), as we did with the training data.
    scaler_test = StandardScaler().fit(df_test)
    scaled_data_test = scaler_test.transform(df_test)

    ##Load in the pre-trained models one by one and run on the data
    for model in trained_models:
        i = 0 
        
        ##Load in classifier model
        loaded_knn = load(model, trusted=True)

        ##Run the loaded classifier on our data. This will give us a label for each data point.
        labeled_test = loaded_knn.predict(scaled_data_test)

        ##Now transform the labels back into the shape of a brain and save as NIFTI.
        reshaped_labels = mask.copy() ##We only want to write back into regions that are 1 on the mask
        reshaped_labels[mask==1] = labeled_test + 1 ##Adding the +1 because labels are 0,1,2... and it gets confusing to look at a 0 region on FSL!
        new_header = img_mask.header.copy() 
        ni_img = nib.Nifti1Image(reshaped_labels, None, header=new_header) 
        nib.save(ni_img, folder_clustering+"new_labels_fuzzc_knn{}.nii.gz".format(cluster_numbers[i])) ##Saving as NIFTI here with the same header as the mask.
    

mkdir: /Users/sharada/Documents/Projects/MDDE/V3/SAMPLE/LOBSTR_P026/clustering_outputs/: File exists


Classifying: LOBSTR_P026


/Users/sharada/Documents/Venvs/mri_venv_basic/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


### Now we have microstructure maps showing different features as different clusters. To ensure consistency in the numbering of clusters and organize it as in the paper (by order of MWF), we need to re-number the clusters carefully.

In [4]:
##Go over all test subjects and re-colour by number.
subject_list = ["LOBSTR_P026"] 

for subject in subject_list:
    print(subject)

    ##Set up folder names
    folder_clustering = base_folder + subject + "/clustering_outputs/" ##Location of clustering output
    folder_mask = base_folder + subject + "/ants/CALIPR/" ##Location of whole brain mask excluding CSF

    ##Load in mask and clustered data
    img_mask = nib.load(folder_mask+"mask_no_csf.nii.gz")
    mask = nib.load(folder_mask+"mask_no_csf.nii.gz").get_fdata()
    clustered = nib.load(folder_clustering + "new_labels_fuzzc_knn6.nii.gz").get_fdata() ##Clustered data
 
    #Wherever the mask is 1, take that metric data into account.
    data_nonzero = clustered[mask== 1]

    ##Re-number the clusters by first calling everything the negative of itself, then replacing it by the correct new number.
    data_nonzero[data_nonzero==1] = -1
    data_nonzero[data_nonzero==2] = -2
    data_nonzero[data_nonzero==3] = -3
    data_nonzero[data_nonzero==4] = -4
    data_nonzero[data_nonzero==5] = -5
    data_nonzero[data_nonzero==6] = -6
    # data_nonzero[data_nonzero==7] = -7
    # data_nonzero[data_nonzero==8] = -8
    # data_nonzero[data_nonzero==9] = -9
    # data_nonzero[data_nonzero==10] = -10

    ##Be careful of this-- it changes depending on training dataset!
    data_nonzero[data_nonzero==-1] = 4+1
    data_nonzero[data_nonzero==-2] = 3+1
    data_nonzero[data_nonzero==-3] = 5+1
    data_nonzero[data_nonzero==-4] = 2+1
    data_nonzero[data_nonzero==-5] = 6+1
    data_nonzero[data_nonzero==-6] = 1+1
    # data_nonzero[data_nonzero==-7] = 0
    # data_nonzero[data_nonzero==-8] = 0
    # data_nonzero[data_nonzero==-9] = 0
    # data_nonzero[data_nonzero==-10] = 0

    changed_clusters = data_nonzero

    ##Now transform the labels back into the shape of a brain and convert it to NIFTI.
    reshaped_labels = mask.copy() ##Only filling in wherever the mask is non-zero
    # Replacing cluster labels
    reshaped_labels[mask==1] = changed_clusters
    new_header = img_mask.header.copy()
    ni_img = nib.Nifti1Image(reshaped_labels, None, header=new_header)
    nib.save(ni_img, folder_clustering+"recoloured_6cluster.nii.gz") ##Writing it to a new NIFTI. 
    !fslmaths {folder_clustering}/recoloured_6cluster.nii.gz -add {folder_mask}/csf_mask.nii.gz {folder_clustering}/recoloured_6cluster.nii.gz ##Adding in the CSF mask so ventricles don't look like a black hole


LOBSTR_P026


### From here on, use recoloured_6cluster.nii.gz as the tissue classification map. This will be consistent cluster numbering for all subjects.

### Also classify the metric atlases (the same as above, but repeated here for completeness with correct file paths for the atlases). This will make the atlas of tissue classification, the target for comparison of all MS people.

In [18]:
## Classifying the atlas uses the same procedure, repeated here for completeness with its correct file paths.
subject_list = ["atlas"]

##The trained model we have saved. Can iterate through a few different models if needed.
trained_models = [base_folder + 'model_6clusters.skops']

##The number of clusters we are going with.
cluster_numbers = [6]

for subject in subject_list:
    ##Set up folder names
    folder_mask = base_folder + subject + "/" ##Location of whole-brain mask excluding CSF
    folder_metrics_mwf = base_folder + subject + "/" ##Location of MWF map
    folder_metrics_tvde = base_folder + subject + "/" ##Location of uFA, CMD maps (registered to MWF space)
    folder_clustering = base_folder + subject + "/clustering_outputs/" ##Output location
    !mkdir {folder_clustering} ##Make the output folder

    mask = nib.load(folder_mask+"mask_no_csf.nii.gz").get_fdata()  ##Whole brain mask excluding CSF
    img_data_1 = nib.load(folder_metrics_tvde+"ufa_atlas.nii.gz").get_fdata()  ##uFA map
    img_data_2 = nib.load(folder_metrics_tvde+"cmd_atlas.nii.gz").get_fdata()  ##C_MD map
    img_data_3 = nib.load(folder_metrics_mwf+"mwf_atlas.nii.gz").get_fdata()  ##MWF map

    ##Separately, hold on to the mask data in this format as we will need its header later on when we generate an image.
    img_mask = nib.load(folder_mask+"mask_no_csf.nii.gz") 

    ##Wherever the mask is 1, take only that metric data into account.
    img_1_nonzero = img_data_1[mask == 1]
    img_2_nonzero = img_data_2[mask == 1]
    img_3_nonzero = img_data_3[mask == 1]

    print("Classifying: " + subject)

    ##Put all metric data into one cohesive dataframe.
    df_test = pd.DataFrame() 
    df_test['img_1'] = img_1_nonzero
    df_test['img_2'] = img_2_nonzero
    df_test['img_3'] = img_3_nonzero

    ##Scale the test dataset (normalize), as we did with the training data.
    scaler_test = StandardScaler().fit(df_test)
    scaled_data_test = scaler_test.transform(df_test)

    ##Load in the pre-trained models one by one and run on the data
    for model in trained_models:
        i = 0 
        
        ##Load in classifier model
        loaded_knn = load(model, trusted=True)

        ##Run the loaded classifier on our data. This will give us a label for each data point.
        labeled_test = loaded_knn.predict(scaled_data_test)

        ##Now transform the labels back into the shape of a brain and save as NIFTI.
        reshaped_labels = mask.copy() ##We only want to write back into regions that are 1 on the mask
        reshaped_labels[mask==1] = labeled_test + 1 ##Adding the +1 because labels are 0,1,2... and it gets confusing to look at a 0 region on FSL!
        new_header = img_mask.header.copy() 
        ni_img = nib.Nifti1Image(reshaped_labels, None, header=new_header) 
        nib.save(ni_img, folder_clustering+"new_labels_fuzzc_knn{}.nii.gz".format(cluster_numbers[i])) ##Saving as NIFTI here with the same header as the mask.
    

        ##Load in mask and clustered data
        img_mask = nib.load(folder_mask+"mask_no_csf.nii.gz")
        mask = nib.load(folder_mask+"mask_no_csf.nii.gz").get_fdata()
        clustered = nib.load(folder_clustering + "new_labels_fuzzc_knn6.nii.gz").get_fdata() ##Clustered data
    
        #Wherever the mask is 1, take that metric data into account.
        data_nonzero = clustered[mask== 1]

        ##Re-number the clusters by first calling everything the negative of itself, then replacing it by the correct new number.
        data_nonzero[data_nonzero==1] = -1
        data_nonzero[data_nonzero==2] = -2
        data_nonzero[data_nonzero==3] = -3
        data_nonzero[data_nonzero==4] = -4
        data_nonzero[data_nonzero==5] = -5
        data_nonzero[data_nonzero==6] = -6
        # data_nonzero[data_nonzero==7] = -7
        # data_nonzero[data_nonzero==8] = -8
        # data_nonzero[data_nonzero==9] = -9
        # data_nonzero[data_nonzero==10] = -10

        ##Be careful of this-- it changes depending on training dataset!
        ##Be careful of this-- it changes depending on training dataset!
        data_nonzero[data_nonzero==-1] = 4+1
        data_nonzero[data_nonzero==-2] = 3+1
        data_nonzero[data_nonzero==-3] = 5+1
        data_nonzero[data_nonzero==-4] = 2+1
        data_nonzero[data_nonzero==-5] = 6+1
        data_nonzero[data_nonzero==-6] = 1+1
        # data_nonzero[data_nonzero==-7] = 0
        # data_nonzero[data_nonzero==-8] = 0
        # data_nonzero[data_nonzero==-9] = 0
        # data_nonzero[data_nonzero==-10] = 0

        changed_clusters = data_nonzero

        ##Now transform the labels back into the shape of a brain and convert it to NIFTI.
        reshaped_labels = mask.copy() ##Only filling in wherever the mask is non-zero
        # Replacing cluster labels
        reshaped_labels[mask==1] = changed_clusters
        new_header = img_mask.header.copy()
        ni_img = nib.Nifti1Image(reshaped_labels, None, header=new_header)
        nib.save(ni_img, folder_clustering+"recoloured_6cluster.nii.gz") ##Writing it to a new NIFTI. 
        ##Adding in the CSF mask so ventricles don't look like a black hole, total number of clusters is then 7 but 1 is just CSF.
        !fslmaths {folder_clustering}/recoloured_6cluster.nii.gz -add {folder_mask}/csf_mask.nii.gz {folder_clustering}/recoloured_6cluster.nii.gz 


mkdir: /Users/sharada/Documents/Projects/MDDE/V3/SAMPLE/atlas/clustering_outputs/: File exists


Classifying: atlas


/Users/sharada/Documents/Venvs/mri_venv_basic/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
